In [269]:
import numpy as np
import matplotlib.pyplot as plt

In [270]:
max_cost = 80
# benefits and volumes must match length of genotypes (currently 40)
benefits = np.array([5, 6, 1, 9, 2, 8, 4, 3, 7, 10, 1, 6, 7, 6, 1, 2, 4, 8, 9, 6, 13,
                     11, 17, 10, 3, 8, 9, 10, 1, 6, 7, 6, 1, 2, 4, 7, 9, 2, 6, 3])
costs = np.array([3, 2, 4, 5, 8, 9, 10, 1, 6, 7, 6, 1, 2, 4, 8, 9, 17, 10, 3, 5,
                  8, 3, 1, 7, 2, 6, 8, 6, 2, 4, 3, 7, 9, 2, 6, 3, 4, 6, 5, 4])

In [271]:
def fitness_function(genotype):
    benefit = []
    cost = []
    for i in range(len(genotype)):
        benefit.append(genotype[i] * benefits[i])
        cost.append(genotype[i] * costs[i])
    if sum(cost) > max_cost:
        return sum(benefit) / (sum(cost) - max_cost + 1)
    return sum(benefit)

In [272]:
def mutate(genotype, prob):
    for i in range(len(genotype)):
        if np.random.uniform(0, 1) < prob:
            # flips gene (0 to 1 or 1 to 0)
            genotype[i] ^= 1

In [273]:
def crossover(w, l, prob):
    for i in range(len(l)):
        if np.random.uniform(0, 1) > prob:
            l[i] = w[i]

In [274]:
def full_microbial_ga(mutation_prob, crossover_prob, generations, k):
    num_genes = 40
    num_individuals = 50
    genotypes = np.zeros((num_individuals, num_genes), dtype=int)
    # genotypes.shape[0] is equal to num_individuals
    fitnesses = np.zeros(genotypes.shape[0])
    max_fitnesses = []
    curr_max_fitness = 0
    for _ in range(generations):
        g1_index = np.random.randint(0, num_individuals)
        g1 = genotypes[g1_index]
        g2_index = np.random.randint(g1_index + 1, g1_index + k) % genotypes.shape[0]
        g2 = genotypes[g2_index]
        if fitnesses[g1_index] >= fitnesses[g2_index]:
            w = g1
            l = g2
            l_index = g2_index
        else:
            w = g2
            l = g1
            l_index = g1_index
        crossover(w, l, crossover_prob)
        mutate(l, mutation_prob)
        genotypes[l_index] = l
        l_fitness = fitness_function(l)
        fitnesses[l_index] = l_fitness
        # Instead of using max(fitnesses) which is a O(n) operation, the below code keeps
        # track of the max fitness through comparison which is more computationally
        # efficient with a complexity of O(1). This is possible since l_fitness is the
        # only newly introduced fitness on each run.
        if l_fitness > curr_max_fitness:
            curr_max_fitness = l_fitness
        max_fitnesses.append(curr_max_fitness)
    return max_fitnesses

In [ ]:
mutation_probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
crossover_prob = 0.5
generations = 100000
k = 3
trials = 10

for mutation_prob in mutation_probs:
    list_max_fitnesses = []
    for trial in range(trials):
        max_fitnesses = full_microbial_ga(mutation_prob, crossover_prob, generations, k)
        list_max_fitnesses.append(max_fitnesses)
    max_fitnesses_means = np.empty(generations)
    max_fitnesses_std = np.empty(generations)
    for g in range(generations):
        max_fitnesses_curr_g = [max_fitnesses[g] for max_fitnesses in list_max_fitnesses]
        max_fitnesses_means[g] = np.mean(max_fitnesses_curr_g)
        max_fitnesses_std[g] = np.std(max_fitnesses_curr_g) / np.sqrt(len(max_fitnesses_curr_g))
    plt.plot(max_fitnesses_means, c='black')
    plt.fill_between(range(generations), max_fitnesses_means - max_fitnesses_std,
                     max_fitnesses_means + max_fitnesses_std, color='gray', alpha=.3)
    plt.xlabel('Generations')
    plt.ylabel('Max Fitness')
    plt.title(f'Max Fitness over Generations, Mutation Probability {mutation_prob}')
    plt.show()

In [ ]:
mutation_probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
crossover_prob = 0.5
generations = 100
k = 3
trials = 10

for mutation_prob in mutation_probs:
    list_max_fitnesses = []
    for trial in range(trials):
        max_fitnesses = full_microbial_ga(mutation_prob, crossover_prob, generations, k)
        list_max_fitnesses.append(max_fitnesses)
    max_fitnesses_means = np.empty(generations)
    max_fitnesses_std = np.empty(generations)
    for g in range(generations):
        max_fitnesses_curr_g = [max_fitnesses[g] for max_fitnesses in list_max_fitnesses]
        max_fitnesses_means[g] = np.mean(max_fitnesses_curr_g)
        max_fitnesses_std[g] = np.std(max_fitnesses_curr_g) / np.sqrt(len(max_fitnesses_curr_g))
    plt.plot(max_fitnesses_means, label=mutation_prob)
    plt.fill_between(range(generations), max_fitnesses_means - max_fitnesses_std,
                     max_fitnesses_means + max_fitnesses_std, alpha=.3)
    print(mutation_prob, max_fitnesses_means[99])
plt.xlabel('Generations')
plt.ylabel('Max Fitness')
plt.title(f'Max Fitness over Generations')
plt.legend(loc="best")
plt.show()

In [ ]:
mutation_probs = ['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%']
# The max fitnesses at generation #100, printed out from the above code and added below
final_max_fitnesses = [97.0, 112.5, 117.1, 112.4, 114.2, 112.0, 103.6, 110.1, 108.2, 107.5]
colors = ['r', 'g', 'b', 'k', 'y']
plt.bar(mutation_probs, final_max_fitnesses, color=colors)
plt.xlabel('Mutation Probability')
plt.ylabel('Fitness')
plt.title('Max Fitness at Generation #100')
plt.show()